In [0]:
# first let's set up access to google sheets and MS.net
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# to MS.net
def loadFromRepo(scriptpath):
    import requests
    url = 'https://raw.githubusercontent.com/MafiaScum-Unofficial/modbot/master/{}'
    url = url.format(scriptpath)
    return requests.get(url).content

exec(loadFromRepo('donbot/donbot.py'))

bot = Donbot(username='Psyche', password='')

In [0]:
# next let's figure out who's /inned
# heuristic will be "posted in the thread but isn't in the notin list Psyche maintains manually"
notin = """Psyche"""

page = html.fromstring(bot.session.get('https://forum.mafiascum.net/viewtopic.php?f=86&t=76586').content)
inned = [p for p in page.xpath('//*[@id="viewtopic"]/fieldset/label[4]/select/option/text()')[1:] if p not in notin]

In [0]:
# update OP
form = {'mode': 'edit', 'f': '86', 'p': 10255537}
startEdit = html.fromstring(bot.session.post('https://forum.mafiascum.net/posting.php?mode=edit&f=86&p=10255537', form).content)

content = startEdit.xpath('//*[@id="message"]/text()')[0]
toreplace = content[content.rfind('[b]Public Opt-Ins:[/b]\n')+len('[b]Public Opt-Ins:[/b]\n'):]
newcontent = content.replace(toreplace, '\n'.join(inned))

form = {'subject': 'SUPP 2018 Anonymous Score Feedback Form',
        'addbbcode20': '100', 'post': 'Submit', 'disable_smilies': 'on',
        'attach_sig': 'on', 'autolock_time': '', 'poll_title': '',
        'poll_option_text': '', 'poll_max_options': '1', 'poll_length': '0',
        'message': newcontent, 
        'lastclick': startEdit.xpath('//input[@name="lastclick"]/@value')[0],
        'edit_post_message_checksum': startEdit.xpath('//input[@name="edit_post_message_checksum"]/@value')[0],
        'edit_post_subject_checksum': startEdit.xpath('//input[@name="edit_post_subject_checksum"]/@value')[0],
        'creation_time': startEdit.xpath('//input[@name="creation_time"]/@value')[0],
        'form_token': startEdit.xpath('//input[@name="form_token"]/@value')[0],
        }

import time
time.sleep(2)
test = bot.session.post('https://forum.mafiascum.net/posting.php?mode=edit&f=86&t=76586&p=10255537', form)

In [12]:
# send a pm for every form response (supposing I've already filtered them!)
sh = gc.open('SUPP Anonymous Score Feedback Form (Responses)')
users = sh.sheet1.col_values('2')[1:]
ratings = sh.sheet1.col_values('3')[1:]
feedback = sh.sheet1.col_values('4')[1:]


tosend = """Someone has provided you feedback through my SUPP Anonymous Score Feedback Form. Here is what they indicated:

User: {}
Rating Volunteered: {}
Explanation Volunteered: "{}"

Note: This message has been automatically generated; it's possible you have already received it and I apologize if that's the case. You are believed to have consented to receiving this message and this message is believed to be both constructive and to follow Site Rules. If you have any concerns about your receipt of this message, feel free to reply."""

for i in range(len(users)):
    print(i, users[i], ratings[i])
    if users[i] not in inned:
        print('{} not found in list of inned users'.format(users[i]))
        continue
    
    if sh.sheet1.cell(str(i+2), 5).value == 'Sent':
        print('{} already sent'.format(i))
        continue
    
    bot.sendPM(sendto=users[i], 
               subject='SUPP Anonymous Score Feedback',
               postdelay=10,
               body=tosend.format(users[i], ratings[i], feedback[i]))
    
    sh.sheet1.update_cell(str(i+2),'5', 'Sent')

0 DeathRowKitty 9
0 already sent
1 Ausuka 8
1 already sent
2 randomidget 10
2 already sent
3 DeathRowKitty 6
3 already sent
4 randomidget 4
4 already sent
5 schadd_ 7
5 already sent
6 Ausuka 3
6 already sent
7 randomidget 3
7 already sent
8 Vijarada 4
8 already sent
9 Davsto 4
9 already sent
10 Shadoweh 8
10 already sent
11 Chickadee 2
11 already sent
12 McMenno 7
12 already sent
13 SleepyKrew 10
13 already sent
14 Shadoweh 10
14 already sent
15 Haschel Cedricson 6
15 already sent
16 xofelf 10
16 already sent
17 Firebringer 0
17 already sent
18 SleepyKrew 10
18 already sent
19 randomidget 6
19 already sent
20 schadd_ 0
20 already sent
21 schadd_ 9
21 already sent
22 hiplop 10
22 already sent
23 DeathRowKitty 10
23 already sent
24 Ausuka 4
24 already sent
25 Vijarada 10
25 already sent
26 Vijarada 10
26 already sent
27 the worst 8-9
27 already sent
28 Ranmaru 2-4
28 already sent
29 the worst 5
29 already sent
30 Errantparabola 9
30 already sent
31 Shadoweh 10
31 already sent
32 Alchemis